In [ ]:
Splunk Indexer SSL - UF
We are configuring SSL communication between Splunk Indexer and Universal Forwarder (UF). This ensures log data is encrypted during transmission.

1. Create Certificate Directory
[root@centos ~]# mkdir /opt/splunk/etc/auth/mycerts
[root@centos ~]# cd /opt/splunk/etc/auth/mycerts/
[root@centos mycerts]# ls -latr
👉 Create a dedicated folder (mycerts) to store SSL certificates.

2. Verify Splunk User Process
[splunk@centos mycerts]$ ps -eo user,comm | grep splunkd | grep -v grep
👉 Confirm that Splunk is running as the splunk user.

3. Check Splunk OS User
[splunk@centos mycerts]$ grep -E '^SPLUNK_OS_USER=' /opt/splunk/etc/splunk-launch.conf
SPLUNK_OS_USER=splunk
👉 Confirms Splunk runs as splunk user (not root).

4. Set Permissions
[root@centos mycerts]# chown -R splunk:splunk /opt/splunk/etc/auth/mycerts
[root@centos mycerts]# chmod 700 /opt/splunk/etc/auth/mycerts
👉 Certificates are owned by splunk and directory access is restricted.

5. Create CA (Certificate Authority)
[root@centos mycerts]# /opt/splunk/bin/splunk cmd openssl genrsa -aes256 -out CAPrivateKey.key 2048
👉 Generate a private key for the CA. This key will be used to sign certificates.

6. Create CSR (Certificate Signing Request) for CA
[root@centos mycerts]# /opt/splunk/bin/splunk cmd openssl req -new -key CAPrivateKey.key -out CACertificate.csr
👉 Generate a CSR for the CA. You’ll enter country, state, organization, etc.

7. Create CA PEM File
[root@centos mycerts]# /opt/splunk/bin/splunk cmd openssl x509 -req -in CACertificate.csr -sha512 -signkey CAPrivateKey.key -CAcreateserial -out CACertificate.pem -days 1095
👉 Sign the CSR with the CA private key to create CACertificate.pem (valid for 3 years).

8. Generate Server Private Key (for Indexer)
[root@centos mycerts]# /opt/splunk/bin/splunk cmd openssl genrsa -aes256 -out ServerPrivateKey.key 2048
👉 Create private key for the indexer server.

9. Create Server CSR
[root@centos mycerts]# /opt/splunk/bin/splunk cmd openssl req -new -key ServerPrivateKey.key -out ServerCertificate.csr
👉 Generate CSR for the server certificate. This will be signed by the CA.

10. Create Server Certificate PEM
[root@centos mycerts]# /opt/splunk/bin/splunk cmd openssl x509 -req -in ServerCertificate.csr -SHA256 -CA CACertificate.pem -CAkey CAPrivateKey.key -CAcreateserial -out ServerCertificate.pem -days 1095
👉 The server certificate is signed using the CA key.

11. Combine Certificates
[root@centos mycerts]# cat ServerCertificate.pem ServerPrivateKey.key CACertificate.pem > SplunkServerCertificate.pem
👉 Merge server cert, private key, and CA cert into one bundle (SplunkServerCertificate.pem).

12. Configure Indexer (inputs.conf)
[SSL]
serverCert = /opt/splunk/etc/auth/mycerts/SplunkServerCertificate.pem
sslPassword = <your_password>
requireClientCert = false
sslVersions = tls1.2,tls1.3

[splunktcp-ssl:9997]
👉 Tells indexer to listen on port 9997 with SSL.

13. Configure Indexer (server.conf)
[sslConfig]
sslRootCAPath = /opt/splunk/etc/auth/mycerts/CACertificate.pem
👉 Defines CA path for SSL validation.

14. Restart Splunk
/opt/splunk/bin/splunk restart
👉 Restart indexer to apply SSL settings.

15. Copy Certificates to UF
[root@centos mycerts]# scp /opt/splunk/etc/auth/mycerts/{SplunkServerCertificate.pem,CACertificate.pem} root@<UF_IP>:/tmp
👉 Transfer required certificates to the Universal Forwarder.

16. Place Certificates on UF
[root@UF tmp]# mkdir /opt/splunkforwarder/etc/auth/mycerts
[root@UF tmp]# mv SplunkServerCertificate.pem CACertificate.pem /opt/splunkforwarder/etc/auth/mycerts/
👉 Store certificates in UF certs directory.

17. Configure UF (outputs.conf)
[tcpout]
defaultGroup = indexergroup
sslPassword = <your_password>

[tcpout:indexergroup]
server = <indexer_ip>:9997
clientCert = /opt/splunkforwarder/etc/auth/mycerts/SplunkServerCertificate.pem
sslVerifyServerCert = true

[tcpout-server://<indexer_ip>:9997]
👉 Defines UF → Indexer SSL communication.

18. Configure UF (server.conf)
[sslConfig]
sslRootCAPath = /opt/splunkforwarder/etc/auth/mycerts/CACertificate.pem
👉 Tells UF where to find CA cert.

19. Configure UF (inputs.conf)
[splunktcp://9997]
disabled = 0
👉 Ensures UF listens properly.

20. Restart UF
/opt/splunkforwarder/bin/splunk restart
21. Verify SSL Connection
On Indexer:

grep -E "Creating fwd data Acceptor.*SSL" /opt/splunk/var/log/splunk/splunkd.log | tail
👉 Check if Indexer accepted SSL forwarder connection.

On Search Head (query internal logs):

index=_internal source=*metrics.log* group=tcpin_connections 
| table _time host version sourceIp destPort ssl fwdType 
| rename ssl as ssl_connection_status fwdType as Forwarder_Type
👉 Confirm that the forwarder is connected using SSL.